In [1]:
import pandas as pd
import zipfile
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [7]:
df_2021 = pd.read_csv(zipfile.ZipFile('raw_data/TRK_13139_FY2021.zip', 'r').open('TRK_13139_FY2021.csv'))
df_2022 = pd.read_csv(zipfile.ZipFile('raw_data/TRK_13139_FY2022.zip', 'r').open('TRK_13139_FY2022.csv'))
df_2023 = pd.read_csv(zipfile.ZipFile('raw_data/TRK_13139_FY2023.zip', 'r').open('TRK_13139_FY2023.csv'))
df_2024_multi = pd.read_csv(zipfile.ZipFile('raw_data/TRK_13139_FY2024_multi_reg.zip', 'r').open('TRK_13139_FY2024_multi_reg.csv'))
df_2024_single = pd.read_csv(zipfile.ZipFile('raw_data/TRK_13139_FY2024_single_reg.zip', 'r').open('TRK_13139_FY2024_single_reg.csv'))

# Shape of each dataframe

In [3]:
df_2021.shape, df_2022.shape, df_2023.shape, df_2024_multi.shape, df_2024_single.shape

((269424, 56), (301447, 56), (474421, 56), (408891, 56), (350103, 56))

# Verify that the datasets have the same column names

In [8]:
cols_2021 = set(df_2021.columns)
cols_2022 = set(df_2022.columns)
cols_2023 = set(df_2023.columns)
cols_2024_multi = set(df_2024_multi.columns)
cols_2024_single = set(df_2024_single.columns)

# Check if all column sets are equal
if cols_2021 == cols_2022 == cols_2023 == cols_2024_multi == cols_2024_single:
    print("Same column names.")
else:
    print("Different column names.")

Same column names.


# Read in and display Data Dictionary

In [ ]:
data_dict = pd.read_excel('preprocessed_data/TRK_13139_I129_H1B_Registrations_FY21_FY24_FOIA_FIN.xlsx', sheet_name = 'Data Dictionary')

# Check difference in shape between original columns listed in data dict and columns that exist in data dict and actual dataset
print(data_dict.loc[:55, :].shape, data_dict.loc[:55, :][lambda df: df['Column'].isin(cols_2021)].shape)

# Set differences between data dict columns and data dict columns existing in H1B dataset
print(f"Data dictionary columns that don't exist in H1B dataset: {set(data_dict.loc[:55, :]['Column']) - set(data_dict.loc[:55, :][lambda df: df['Column'].isin(cols_2021)]['Column'])}")
print(f"H1B Dataset columns that do not exist in data dictionary: {set(df_2021.columns) - set(data_dict.loc[:55, :]['Column'])}")
print()
# Print out the columns that do not exist in data dictionary:
print("H1B Dataset columns that do not exist in data dictionary:")
for col in cols_2021:
    if col not in np.array(data_dict.loc[:55, :]['Column']):
        print(f"    {col}")

# Seems like that they do exist but data dict cols are uppercase and dataset cols are lowercase.
data_dict.loc[:55, :][lambda df: df['Column'].isin(cols_2021)].sort_values(by = 'Column')

(56, 2) (52, 2)
Data dictionary columns that don't exist in H1B dataset: {'VALID_TO', 'FIRST_DECISION_DATE', 'REC_DATE', 'VALID_FROM'}
H1B Dataset columns that do not exist in data dictionary: {'valid_to', 'valid_from', 'first_decision_date', 'rec_date'}

H1B Dataset columns that do not exist in data dictionary:
    rec_date
    valid_to
    first_decision_date
    valid_from


,Column,Description
27,BASIS_FOR_CLASSIFICATION,"Basis for Classification, as provided by the p..."
50,BEN_COMP_PAID,Beneficiary's rate of pay per year
31,BEN_COUNTRY_OF_BIRTH,Beneficiary's country of birth
32,BEN_CURRENT_CLASS,Beneficiary's current classification
47,BEN_EDUCATION_CODE,Beneficiary's education code
49,BEN_PFIELD_OF_STUDY,Beneficiary's field of study
30,BEN_SEX,Beneficiary's gender
34,DOL_ETA_CASE_NUMBER,Unique identifier for the DOL ETA Case.
51,DOT_CODE,Beneficiary's job code\nRefer to descriptions ...
48,ED_LEVEL_DEFINITION,Beneficiary's education definition (descript t...


In [26]:
col_names = data_dict.loc[:55, :][lambda df: df['Column'].isin(cols_2021)].to_dict(orient = 'list')['Column']
descriptions = data_dict.loc[:55, :][lambda df: df['Column'].isin(cols_2021)].to_dict(orient = 'list')['Description']

for i in range(len(col_names)):
    print(f"({i+1}): {col_names[i]}: {descriptions[i]}" )

(1): bcn: Beneficiary confirmation number. Unique ID for each registration submitted
(2): country_of_birth: Beneficiary's country of birth
(3): country_of_nationality: Beneficiary's country of nationality
(4): ben_date_of_birth: Beneficiary's date of birth
(5): ben_year_of_birth: Beneficiary's year of birth
(6): gender: Beneficiary's gender
(7): employer_name: Employer name for registration
(8): FEIN: Employer's tax number
(9): mail_addr: Employer's mailing address
(10): city: Employer's city
(11): state: Employer's state
(12): zip: Employer's zip code
(13): agent_first_name: First name of registered agent
(14): agent_last_name: Last name of registered agent
(15): lottery_year: Lottery year of registration
(16): status_type: Status of the registration. "Selected" means the registration was selected in the lottery. "Created" or "Eligible" means the registration was not selected in the lottery.
(17): ben_multi_reg_ind: Indicator of multiple registrations for beneficiary (a value of 1 rep

# What is the difference between multi and single registrations in 2024?

# Q: Which factors contribute most to the approval / denial of a H1B petition once it has been selected from the lottery?
Applicants are only approved or denied from an H1B petition after they were selected from the lottery. Therefore we are only interested in observations that have the **"Selected"** status type.

In [43]:
selected_status_condition = lambda df: df['status_type'] == 'SELECTED'

df_2021 = df_2021[selected_status_condition]
df_2022 = df_2022[selected_status_condition]
df_2023 = df_2023[selected_status_condition]
df_2024_multi = df_2024_multi[selected_status_condition]
df_2024_single = df_2024_single[selected_status_condition]

# Print new shapes of each dataset after filtering
df_2021.shape, df_2022.shape, df_2023.shape, df_2024_multi.shape, df_2024_single.shape

((124368, 56), (131896, 56), (127563, 56), (103060, 56), (85304, 56))

In [ ]:
# Write intermediate datasets to CSV files
df_2021.to_csv('preprocessed_data/FY2021_cleaned_H1B.csv')
df_2022.to_csv('preprocessed_data/FY2022_cleaned_H1B.csv')
df_2023.to_csv('preprocessed_data/FY2023_cleaned_H1B.csv')
df_2024_multi.to_csv('preprocessed_data/FY2024_multi_reg_cleaned_H1B.csv')
df_2024_single.to_csv('preprocessed_data/FY2024_single_reg_cleaned_H1B.csv')

# Come up with columns of interest via intuition

**FIRST_DECISION** (outcome Y): The first decision of the petition, which indicates whether it was approved or denied.

**FIRST_DECISION_DATE**: The date of the first decision, which might provide insights into processing times.

**employer_name**: The name of the employer, which could be relevant in understanding the employer's influence on the decision.

**JOB_TITLE**: The job title of the beneficiary, which can indicate the type of job being offered.

**WAGE_AMT**: The wage amount offered to the beneficiary, which is a critical factor in the decision.

**WORKSITE_STATE**: The state where the beneficiary will work, as certain locations might have different approval rates.

**BEN_EDUCATION_CODE**: The education level of the beneficiary, which might influence the decision.

**BEN_PFIELD_OF_STUDY**: The field of study of the beneficiary, which can be relevant to the job being offered.

**FULL_TIME_IND**: Indicator for a full-time role, which might affect the decision.

**NUM_OF_EMP_IN_US**: The number of employees in the US for the petitioner, which might indicate the size and stability of the employer.

In [7]:
cols_of_interest = [
    'employer_name',
    'JOB_TITLE',
    'WAGE_AMT',
    'WORKSITE_STATE',
    'BEN_EDUCATION_CODE',
    'BEN_PFIELD_OF_STUDY',
    'FULL_TIME_IND',
    'NUM_OF_EMP_IN_US',
    'FIRST_DECISION_DATE',
    'FIRST_DECISION'
]

**Your slides should include**:
- Title, Authors

- What is the question you will be working on? Why is it interesting?


- What is the data you will be using? Include data source, size of dataset, main features to be used. Please also include summary statistics of your data.


- What prediction algorithms do you plan to use? Please describe them in detail?


- How will you evaluate your results? Please describe your chosen performance metrics and/or statistical tests in detail